[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/notebooks/blob/main/camenduru's_discord_diffusers.ipynb.ipynb)

In [ ]:
!pip install -qq https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.14/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl
!pip install -U -qq  git+https://github.com/camenduru/diffusers transformers

In [ ]:
import torch, os, gc, requests, json
from diffusers import StableDiffusionPipeline
from PIL.PngImagePlugin import PngInfo

metadata = PngInfo()

def closestNumber(n, m):
    q = int(n / m)
    n1 = m * q
    if (n * m) > 0:
        n2 = m * (q + 1)
    else:
        n2 = m * (q - 1)
    if abs(n - n1) < abs(n - n2):
        return n1
    return n2

is_tile = False #@param {type: 'boolean'}
if is_tile:
    def patch_conv(cls):
        init = cls.__init__
        def __init__(self, *args, **kwargs):
            return init(self, *args, **kwargs, padding_mode="circular")
        cls.__init__ = __init__
    patch_conv(torch.nn.Conv2d)

model_hash = "none" #@param {type: 'string'}
max_files = 500 #@param {type: 'string'}
root_folder = "images" #@param {type: 'string'}
model_name = "hakurei/waifu-diffusion" #@param ["nitrosocke/mo-di-diffusion", "hakurei/waifu-diffusion"] {allow-input: true}
pipe = StableDiffusionPipeline.from_pretrained(model_name, safety_checker=None).to("cuda")

if os.path.exists(f"{root_folder}") == False:
    os.mkdir(f"{root_folder}")
image_folder = max([int(f) for f in os.listdir(f"{root_folder}")], default=0)
if os.path.exists(f"{root_folder}/{image_folder:04}") == False:
    os.mkdir(f"{root_folder}/{image_folder:04}")
name = max([int(f[: f.index(".")]) for f in os.listdir(f"{root_folder}/{image_folder:04}")],default=0,)

def generate(discord_token, discord_channel_id, by, num_inference_steps, guidance_scale, sampler, width, height, prompt, image_folder, name):
    width = closestNumber(width, 8)
    height = closestNumber(height, 8)
    metadata.add_text("Prompt", f"{prompt}")
    metadata.add_text("by", f"{by}")
    gc.collect()
    torch.cuda.empty_cache()
    image = pipe([prompt], height=height, width=width, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale).images[0]
    initial_seed = torch.cuda.initial_seed()
    image.save(f"{root_folder}/{image_folder:04}/{name:04}.png", pnginfo=metadata)
    files = {f"{image_folder:04}_{name:04}.png": open(f"{root_folder}/{image_folder:04}/{name:04}.png", "rb").read()}
    payload = {"content": f"{prompt}\nSteps: {num_inference_steps}, Sampler: {sampler}, CFG scale: {guidance_scale}, Seed: {initial_seed}, Size: {width}x{height}, Model hash: {model_hash}, Model name: {model_name}"}
    requests.post(f"https://discord.com/api/v9/channels/{discord_channel_id}/messages", data=payload, headers={"authorization": f"Bot {discord_token}"}, files=files)

while True:
    if name < max_files:
        with open("prompts.json", "r") as file:
            prompts = file.readlines()
        for prompt in prompts:
            d = json.loads(prompt)
            name += 1
            generate(d["discord_token"], d["discord_channel_id"], d["by"], d["num_inference_steps"], d["guidance_scale"], d["sampler"], d["width"], d["height"], d["prompt"], image_folder, name)
    else:
        image_folder += 1
        if os.path.exists(f"{root_folder}/{image_folder:04}") == False:
            os.mkdir(f"{root_folder}/{image_folder:04}")
        name = 0